# Photometric Realization from Different Magnitude Error Models

author: John Franklin Crenshaw, Sam Schmidt, Eric Charles, Ziang Yan

last run successfully: Feb 9, 2026

This notebook demonstrates how to do photometric realization from different magnitude error models. For more completed degrader demo, see `00_Quick_Start_in_Creation.ipynb`

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pzflow
import rail.interactive as ri

Specify the path to the pretrained 'pzflow' used to generate samples

In [ ]:
flow_file = os.path.join(
    os.path.dirname(pzflow.__file__), "example_files", "example-flow.pzflow.pkl"
)

### "True" Engine

First, let's make an Engine that has no degradation. We can use it to generate a "true" sample, to which we can compare all the degraded samples below.

Note: in this example, we will use a normalizing flow engine from the [pzflow](https://github.com/jfcrenshaw/pzflow) package. However, everything in this notebook is totally agnostic to what the underlying engine is.

In [ ]:
n_samples = int(1e5)

samples_truth = ri.creation.engines.flowEngine.flow_creator(
    n_samples=n_samples, model=flow_file, seed=0
)

In [ ]:
samples_truth["output"]["major"] = np.abs(
    np.random.normal(loc=0.01, scale=0.1, size=n_samples)
)  # add major and minor axes
b_to_a = 1 - 0.5 * np.random.rand(n_samples)
samples_truth["output"]["minor"] = samples_truth["output"]["major"] * b_to_a

samples_truth["output"]


### LSSTErrorModel

Now, we will demonstrate the `LSSTErrorModel`, which adds photometric errors using a model similar to the model from [Ivezic et al. 2019](https://arxiv.org/abs/0805.2366) (specifically, it uses the model from this paper, without making the high SNR assumption. To restore this assumption and therefore use the exact model from the paper, set `highSNR=True`.)

Let's create an error model with the default settings for point sources:

In [ ]:
samples_w_errs = ri.creation.degraders.photometric_errors.lsst_error_model(
    sample=samples_truth["output"]
)

For extended sources:

In [ ]:
samples_w_errs_auto = ri.creation.degraders.photometric_errors.lsst_error_model(
    sample=samples_truth["output"], extendedType="auto"
)

In [ ]:
samples_w_errs_gaap = ri.creation.degraders.photometric_errors.lsst_error_model(
    sample=samples_truth["output"], extendedType="gaap"
)

Notice some of the magnitudes are inf's.
These are non-detections (i.e. the noisy flux was negative).
You can change the nSigma limit for non-detections by setting `sigLim=...`.
For example, if `sigLim=5`, then all fluxes with `SNR<5` are flagged as non-detections.

Let's plot the error as a function of magnitude

In [ ]:
fig, axes_ = plt.subplots(ncols=3, nrows=2, figsize=(15, 9), dpi=100)
axes = axes_.reshape(-1)
for i, band in enumerate("ugrizy"):
    ax = axes[i]
    # pull out the magnitudes and errors
    mags = samples_w_errs["output"][band].to_numpy()
    errs = samples_w_errs["output"][band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    # ax.plot(mags, errs, label=band)

    # plt.plot(mags, errs, c='C'+str(i))
    ax.scatter(
        samples_w_errs_gaap["output"][band].to_numpy(),
        samples_w_errs_gaap["output"][band + "_err"].to_numpy(),
        s=5,
        marker=".",
        color="C0",
        alpha=0.8,
        label="GAAP",
    )

    ax.plot(mags, errs, color="C3", label="Point source")

    ax.legend()
    ax.set_xlim(18, 31)
    ax.set_ylim(-0.1, 3.5)
    ax.set(xlabel=band + " Band Magnitude (AB)", ylabel="Error (mags)")

In [ ]:
fig, axes_ = plt.subplots(ncols=3, nrows=2, figsize=(15, 9), dpi=100)
axes = axes_.reshape(-1)
for i, band in enumerate("ugrizy"):
    ax = axes[i]
    # pull out the magnitudes and errors
    mags = samples_w_errs["output"][band].to_numpy()
    errs = samples_w_errs["output"][band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    # ax.plot(mags, errs, label=band)

    # plt.plot(mags, errs, c='C'+str(i))
    ax.scatter(
        samples_w_errs_auto["output"][band].to_numpy(),
        samples_w_errs_auto["output"][band + "_err"].to_numpy(),
        s=5,
        marker=".",
        color="C0",
        alpha=0.8,
        label="AUTO",
    )

    ax.plot(mags, errs, color="C3", label="Point source")

    ax.legend()
    ax.set_xlim(18, 31)
    ax.set_ylim(-0.1, 3.5)
    ax.set(xlabel=band + " Band Magnitude (AB)", ylabel="Error (mags)")

You can see that the photometric error increases as magnitude gets dimmer, just like you would expect, and that the extended source errors are greater than the point source errors.
The extended source errors are also scattered, because the galaxies have random sizes.

Also, you can find the GAaP and AUTO magnitude error are scattered due to variable galaxy sizes. Also, you can find that there are gaps between GAAP magnitude error and point souce magnitude error, this is because the additional factors due to aperture sizes have a minimum value of $\sqrt{(\sigma^2+A_{\mathrm{min}})/\sigma^2}$, where $\sigma$ is the width of the beam, $A_{\min}$ is an offset of the aperture sizes (taken to be 0.7 arcmin here).

You can also see that there are *very* faint galaxies in this sample.
That's because, by default, the error model returns magnitudes for all positive fluxes.
If you want these galaxies flagged as non-detections instead, you can set e.g. `sigLim=5`, and everything with `SNR<5` will be flagged as a non-detection.